# Quasilikelihood and semiparametric methods for the general linear model

In [1]:
import collections
import enum
import functools
import itertools
from multiprocessing import pool as pool_lib
from typing import Callable, List, NamedTuple, Sequence, Tuple

import numpy as np
import pandas as pd
from scipy import stats
from scipy import linalg

BETA = np.array([0., 0.5], dtype=np.float64)
DESIGN_CLUSTERS = collections.OrderedDict([
    ('I', [[7, 10, 13, 16]]),
    ('II', [[7, 10, 13], [7, 10, 16], [7, 13, 16], [10, 13, 16]])
])
NUM_CLUSTERS = [15, 30, 60]
WITHIN_CLUSTER_CORRELATIONS = [0.5, 0.9]

CorrelationStructure = enum.Enum(
    'CorrelationStructure',
    'NONE EXCHANGEABLE EXPONENTIAL')

EstimationMethod = enum.Enum(
    'EstimationMethod',
    'GLS QL Sandwich')

In [2]:
class Experiment(NamedTuple('Experiment', [
    ('beta', np.array),
    ('error_variance', float),
    ('num_clusters', Sequence[Tuple[np.array, np.array]]),
    ('clusters', Sequence[np.array]),
    ('within_cluster_correlation', float),
    ('within_cluster_correlation_structure', CorrelationStructure),
])):
    """Encapsulates parameters for the data generating mechanism."""
    
    def sample_clusters(self) -> List[Tuple[np.array, np.array]]:
        return [self._sample_cluster() for _ in range(self.num_clusters)]
    
    def _sample_cluster(self) -> Tuple[np.array, np.array]:
        covariates = self._sample_cluster_covariates()
        covariates = np.column_stack((np.ones(len(covariates)), covariates))
        covariance = self._make_within_cluster_covariance(len(covariates))
        response = stats.multivariate_normal(
            mean=np.matmul(covariates, self.beta), cov=covariance).rvs()
        return covariates, response
    
    def _sample_cluster_covariates(self) -> np.array:
        return self.clusters[np.random.choice(len(self.clusters))]
    
    def _make_within_cluster_covariance(self, cluster_size):
        correlation = np.eye(cluster_size)
        if self.within_cluster_correlation_structure == CorrelationStructure.EXCHANGEABLE:
            correlation[correlation == 0] = self.within_cluster_correlation
        elif self.within_cluster_correlation_structure == CorrelationStructure.EXPONENTIAL:
            for i in range(cluster_size):
                for j in range(i + 1, cluster_size):
                    correlation[i, j] = correlation[j, i] = np.power(
                        self.within_cluster_correlation, np.abs(j - i))
        return self.error_variance*correlation
        
    @classmethod
    def from_template(
        cls,
        clusters,
        num_clusters,
        within_cluster_correlation,
        within_cluster_correlation_structure) -> 'Experiment':
        assert len(set([len(cluster) for cluster in clusters])) == 1,\
               'Clusters must be the same size.'
        
        return cls(beta=BETA,
                   clusters=clusters,
                   error_variance=1.,
                   num_clusters=num_clusters,
                   within_cluster_correlation=within_cluster_correlation,
                   within_cluster_correlation_structure=within_cluster_correlation_structure)

In [3]:
def sum_dict(acc, result):
    if type(acc) == dict:        
        return {key: sum_dict(value, result[key]) for key, value in acc.items()}   
    return acc + result
    
def divide_dict(results, d):
    if type(results) == dict:
        return {key: divide_dict(value, d) for key, value in results.items()}
    return results/d

In [4]:
def estimate_rho(epsilon_hat):
    covariance = np.outer(epsilon_hat, epsilon_hat)    
    rho_exchangeable = 0.
    rho_exponential = 0.
    for i in range(len(covariance)):
        for j in range(i + 1, len(covariance[i])):
            rho_exchangeable += covariance[i, j]
            if j - i == 1:
                rho_exponential += covariance[i, j]
    rho_exchangeable /= (np.square(covariance.shape[0]) - covariance.shape[0])/2
    rho_exponential /= covariance.shape[0] - 1
    return rho_exchangeable, rho_exponential

def make_correlation_matrices(clusters, beta_hat, sigma_2_hat):
    rho_exchangeable = 0.
    rho_exponential = 0.        
    for X, y in clusters:
        cluster_rho_exchangeable, cluster_rho_exponential = estimate_rho(
            (y - X.dot(beta_hat))/np.sqrt(sigma_2_hat))
        rho_exchangeable += cluster_rho_exchangeable
        rho_exponential += cluster_rho_exponential
            
    rho_exchangeable /= len(clusters)
    rho_exponential /= len(clusters)
    
    correlation_matrices = []
    for X, y in clusters:
        exchangeable_matrix = np.eye(len(y))
        exchangeable_matrix[exchangeable_matrix == 0] = rho_exchangeable
        exponential_matrix = np.eye(len(y))
        for i in range(len(y) - 1):
            for j in range(i + 1, len(y)):
                exponential_matrix[i, j] = exponential_matrix[j, i] = np.power(rho_exponential, j - i)
        correlation_matrices.append({
            CorrelationStructure.NONE.name: np.eye(len(y)),
            CorrelationStructure.EXCHANGEABLE.name: exchangeable_matrix,
            CorrelationStructure.EXPONENTIAL.name: exponential_matrix,            
        })
        
    return correlation_matrices

def estimate_beta_hats(clusters, correlation_matrices):
    def project(X, y, weight):                
        return {
            'X': X.T.dot(weight).dot(X),
            'y': X.T.dot(weight).dot(y),
        }
    
    projections = functools.reduce(sum_dict, [
        {
            key: project(X, y, linalg.cho_solve(
                linalg.cho_factor(correlation_matrix), np.eye(len(correlation_matrix))))
            for key, correlation_matrix in matrix_dict.items()
        } 
        for (X, y), matrix_dict in zip(clusters, correlation_matrices)
    ])
        
    return {
        key: linalg.cho_solve(linalg.cho_factor(projection['X']), projection['y'])
        for key, projection in projections.items()
    }

def estimate_covariance(clusters,
                        correlation_matrices,
                        method,
                        beta_hat):
    if method != EstimationMethod.Sandwich:
        covariance = np.zeros((len(beta_hat), len(beta_hat)))
        dispersion_factor = 0.
        total = 0.
        for (X, y), correlation_matrix in zip(clusters, correlation_matrices):
            weight = linalg.cho_solve(
                linalg.cho_factor(correlation_matrix), np.eye(len(correlation_matrix)))
            covariance += X.T.dot(weight).dot(X)
            dispersion_factor += np.sum(np.square(y - X.dot(beta_hat)))
            total += len(y)
        covariance = linalg.cho_solve(linalg.cho_factor(covariance), np.eye(len(beta_hat)))
        dispersion_factor /= total - len(beta_hat)
        return covariance if method == EstimationMethod.GLS else covariance*dispersion_factor
    # Sandwich estimation.
    bread = np.zeros((len(beta_hat), len(beta_hat)))
    meat = np.zeros((len(beta_hat), len(beta_hat)))
    for (X, y), correlation_matrix in zip(clusters, correlation_matrices):
        weight = linalg.cho_solve(
                linalg.cho_factor(correlation_matrix), np.eye(len(correlation_matrix)))
        bread += X.T.dot(weight).dot(X)        
        epsilon_hat = y - X.dot(beta_hat)
        meat += X.T.dot(weight).dot(np.outer(epsilon_hat, epsilon_hat)).dot(weight).dot(X)        
    bread = linalg.cho_solve(linalg.cho_factor(bread), np.eye(len(beta_hat)))    
    return bread.dot(meat).dot(bread)
    
def run_experiment(experiment, estimate_beta=False):
    clusters = experiment.sample_clusters()
    X = np.vstack([X for X, _ in clusters])
    y = np.hstack([y for _, y in clusters])
    
    gram_matrix_ols = X.T.dot(X)
    
    beta_hat_ols = linalg.cho_solve(linalg.cho_factor(gram_matrix_ols), X.T.dot(y))
    sigma_2_hat_ols = np.sum(np.square(y - X.dot(beta_hat_ols)))/(len(y) - len(beta_hat_ols))
    
    correlation_matrices = make_correlation_matrices(
        clusters,
        beta_hat_ols,
        sigma_2_hat_ols)
    beta_hats = estimate_beta_hats(clusters, correlation_matrices)
    
    if estimate_beta:
        return beta_hats
    
    return {
        method.name: {
            correlation_structure: np.sqrt(estimate_covariance(
                clusters,
                [matrix_dict[correlation_structure] for matrix_dict in correlation_matrices],
                method,
                beta_hat)[1, 1])
            for correlation_structure, beta_hat in beta_hats.items()
        }
        for method in EstimationMethod
    }

In [5]:
def run_experiments(experiment, num_trials):
    pool = pool_lib.Pool(4)
    results = pool.map(run_experiment, [experiment]*num_trials)
    pool.close()
    results = functools.reduce(sum_dict, results)
    return divide_dict(results, num_trials)

In [6]:
experiments = [
    Experiment.from_template(design, num_clusters, correlation_coefficient, correlation_structure)
    for design, num_clusters, correlation_coefficient, correlation_structure
    in
    itertools.product(
       [DESIGN_CLUSTERS['I'], DESIGN_CLUSTERS['II']],
       NUM_CLUSTERS,
       WITHIN_CLUSTER_CORRELATIONS,
       [CorrelationStructure.EXCHANGEABLE, CorrelationStructure.EXPONENTIAL])
]

In [7]:
def index_experiment(experiment):    
    return (experiment.num_clusters,
            [k for k, v in DESIGN_CLUSTERS.items() if experiment.clusters == v][0],
            experiment.within_cluster_correlation_structure.name,
            experiment.within_cluster_correlation)

simulation_results = pd.DataFrame(
    index=pd.MultiIndex.from_product(
        [NUM_CLUSTERS, DESIGN_CLUSTERS.keys(),
         [CorrelationStructure.EXCHANGEABLE.name, CorrelationStructure.EXPONENTIAL.name],
         WITHIN_CLUSTER_CORRELATIONS,         
        ],
        names=['$n$', 'Design', 'Correlation structure', 'Correlation']),
    columns=pd.MultiIndex.from_product(
        [[value.name for value in EstimationMethod],
         [value.name for value in CorrelationStructure]],
        names=['Estimator', 'Assumed correlation']
    ))

In [8]:
for experiment in experiments:
    simulation_results.loc[index_experiment(experiment)] = (
        pd.DataFrame.from_dict(run_experiments(experiment, 4096), orient='index').stack())
simulation_results

Estimator                                           GLS               \
Assumed correlation                                NONE EXCHANGEABLE   
$n$ Design Correlation structure Correlation                           
15  I      EXCHANGEABLE          0.5            0.03849     0.028486   
                                 0.9            0.03849    0.0146635   
           EXPONENTIAL           0.5            0.03849    0.0317356   
                                 0.9            0.03849     0.017354   
    II     EXCHANGEABLE          0.5          0.0447223    0.0345043   
                                 0.9          0.0447223    0.0190387   
           EXPONENTIAL           0.5          0.0447223    0.0365402   
                                 0.9          0.0447223    0.0206638   
30  I      EXCHANGEABLE          0.5          0.0272166    0.0197434   
                                 0.9          0.0272166   0.00953959   
           EXPONENTIAL           0.5          0.0272166     0.022187   
                                 0.9          0.0272166    0.0115721   
    II     EXCHANGEABLE          0.5          0.0315212     0.023777   
                                 0.9          0.0315212     0.012012   
           EXPONENTIAL           0.5          0.0315212    0.0253823   
                                 0.9          0.0315212    0.0132897   
60  I      EXCHANGEABLE          0.5           0.019245    0.0137719   
                                 0.9           0.019245     0.006412   
           EXPONENTIAL           0.5           0.019245    0.0155736   
                                 0.9           0.019245   0.00792484   
    II     EXCHANGEABLE          0.5          0.0222478    0.0166101   
                                 0.9          0.0222478   0.00797744   
           EXPONENTIAL           0.5          0.0222478    0.0178022   
                                 0.9          0.0222478   0.00894615   

Estimator                                                        QL  \
Assumed correlation                          EXPONENTIAL       NONE   
$n$ Design Correlation structure Correlation                          
15  I      EXCHANGEABLE          0.5           0.0378713  0.0376096   
                                 0.9           0.0246101    0.03698   
           EXPONENTIAL           0.5           0.0379941   0.037663   
                                 0.9           0.0246015  0.0370272   
    II     EXCHANGEABLE          0.5           0.0402881  0.0440769   
                                 0.9           0.0250425  0.0435155   
           EXPONENTIAL           0.5           0.0402493  0.0441191   
                                 0.9           0.0249189  0.0435549   
30  I      EXCHANGEABLE          0.5           0.0268184  0.0267877   
                                 0.9            0.016304  0.0265356   
           EXPONENTIAL           0.5           0.0268626  0.0267972   
                                 0.9           0.0162379  0.0265477   
    II     EXCHANGEABLE          0.5           0.0281939  0.0312162   
                                 0.9           0.0159787   0.031043   
           EXPONENTIAL           0.5           0.0281831  0.0312246   
                                 0.9           0.0159189  0.0310562   
60  I      EXCHANGEABLE          0.5           0.0189363   0.019091   
                                 0.9           0.0110274  0.0190104   
           EXPONENTIAL           0.5           0.0189491  0.0191008   
                                 0.9            0.011011  0.0190175   
    II     EXCHANGEABLE          0.5           0.0198285  0.0221211   
                                 0.9           0.0106526   0.022059   
           EXPONENTIAL           0.5           0.0198292  0.0221256   
                                 0.9           0.0106462  0.0220642   

Estimator                                                              \
Assumed correlation                          EXCHANGEABLE EXPONE

In [9]:
import os

if not os.path.isdir('simulation_results'):
    os.mkdir('simulation_results')

for key, values in simulation_results.iterrows():
    file_name = '-'.join(map(str, key)).replace('.', '_')
    with open('simulation_results/{}.tex'.format(file_name), 'w') as f:
        f.write(' & '.join(map(lambda v: str(np.round(v, decimals=5)), values.values)))